# Find cases when proteoforms clarify gene-level overlap

### Set up configuration

In [7]:


import config
from lib.graph_database_access import get_query_result
from queries import QUERY_GET_NUM_PROTEOFORMS_PER_PROTEIN, QUERY_GET_REACTIONS_BY_PROTEIN_FOR_MULTIPROTEOFORM_PROTEINS

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

config.set_root_wd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initial working directory: C:\git\ProteoformNetworks\src\Python
New working directory: C:\git\ProteoformNetworks


In [8]:
## Identify proteins with at least a proteoform in one set of reactions and another in a different set of reactions.

## Get Reactions by proteoform

## Proteins with 2+ proteoforms, 2+ reactions and 1 proteoform being the only one participating in a reaction.

df_proteins = get_query_result(QUERY_GET_REACTIONS_BY_PROTEIN_FOR_MULTIPROTEOFORM_PROTEINS)
print(f"There are {len(df_proteins)} proteins with multiple proteoforms.")


There are 1493 proteins with multiple proteoforms.


In [11]:
# Which one of these 1493 possibilities, has dissociate reaction sets in their proteoforms.
# Each protein has n sets of reactions.
# Find a protein where at least one set has no intersection with the other sets.

def isExample(row):
    l = list(map(lambda reactionSet: set(reactionSet), row["ReactionSets"]))
    for i in range(row["NumProteoforms"]):
        left = []
        right = []
        if i == 0:
            right = l[1:]
        elif i == len(l):
            left = l[:len(l)-1]
        else:
            left = l[:i]
            right = l[i+1:]
        #print(f"Compare: {l[i]} with: {left + right}")
        intersection = l[i].intersection(*(left + right))
        if len(intersection) == 0:
            # print("++++++++++++++++++++++++++++++++++++++++++++")
            # print(f"Proteoform {i+1} ({row['Proteoforms'][i]}) that participantes in: {l[i]}")
            # print(row["Proteoforms"])
            # print(row["ReactionSets"])
            return True;
    return False;
selection = df_proteins.apply(lambda row : isExample(row), axis = 1)
examples = df_proteins.loc[selection][["Proteoforms", "PhysicalEntitySets", "ReactionSets"]]
examples

,Proteoforms,PhysicalEntitySets,ReactionSets
0,"[[P36969, 00031:73], [P36969-2]]","[[R-HSA-2142826], [R-HSA-9020572]]","[[R-HSA-2161791, R-HSA-2161999, R-HSA-2161959,..."
20,"[[P04279, 00034:239], [P04279]]","[[R-HSA-6810688], [R-HSA-977100]]","[[R-HSA-6810643], [R-HSA-977136]]"
112,"[[Q15080, 00046:315, 00047:154], [Q15080]]","[[R-HSA-1996221], [R-HSA-1222707]]","[[R-HSA-9014424, R-HSA-1236967, R-HSA-5668605,..."
169,"[[Q9BX63, 00046:990, 00064:1249], [Q9BX63]]","[[R-HSA-5687710], [R-HSA-2564781]]","[[R-HSA-9701199, R-HSA-9709601, R-HSA-5686657,..."
182,"[[P54840, 00046:null], [P54840]]","[[R-HSA-71604], [R-HSA-3858521, R-HSA-3858519,...","[[R-HSA-3322019], [R-HSA-3858506, R-HSA-378099..."
...,...,...,...
1488,"[[P0CG47, 00134:76], [P0CG47, 00134:152], [P0C...","[[R-HSA-8869049], [R-HSA-8869047], [R-HSA-8869...","[[R-HSA-5690319], [R-HSA-5690319], [R-HSA-5690..."
1489,"[[P04637, 00046:15, 00046:20, 00046:46], [P046...","[[R-HSA-6798369], [R-HSA-3222124], [R-HSA-3222...","[[R-HSA-6799409, R-HSA-6798374, R-HSA-6798611,..."
1490,"[[Q71DI3, 00046:11, 00047:12, 00064:15, 00064:...","[[R-HSA-8871289], [R-HSA-8871296], [R-HSA-8865...","[[R-HSA-8865613], [R-HSA-8865613], [R-HSA-8865..."
1491,"[[P68431, 00046:11, 00047:12, 00064:15, 00064:...","[[R-HSA-8871286], [R-HSA-8871295], [R-HSA-8865...","[[R-HSA-8865613], [R-HSA-8865613], [R-HSA-8865..."


In [10]:
print(f"Found {len(examples)} proteins that have at least one proteoform participating in separate reactions.")

Found 497 proteins that have at least one proteoform participating in separate reactions.
